In [4]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold

from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso, Ridge

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
train_matches = pd.read_csv('train.csv')
test_matches = pd.read_csv('test.csv')
gold = pd.read_csv('gold.csv')
heroes = pd.read_csv('heroes.csv')
items = pd.read_csv('items.csv')
lh = pd.read_csv('lh.csv')
xp = pd.read_csv('xp.csv')
events = pd.read_csv('events.csv')

IOError: File train.csv does not exist

In [4]:
print 'train_matches.shape', train_matches.shape
print 'test_matches.shape', test_matches.shape
print 'gold.shape', gold.shape
print 'heroes.shape', heroes.shape
print 'items.shape', items.shape
print 'lh.shape', lh.shape
print 'xp.shape', xp.shape
print 'events.shape', events.shape

train_matches.shape (24974, 2)
test_matches.shape (24974, 1)
gold.shape (499480, 12)
heroes.shape (49948, 11)
items.shape (499412, 123)
lh.shape (499480, 12)
xp.shape (499480, 12)
events.shape (69120, 4)


#### Что такое event

In [17]:
for i in np.unique(events[['event_type']]):
    print i, events['event_type'].value_counts()[i]
events.head()

0 953
1 12
2 17
3 48655
4 968
5 2251
6 16264


,mid,event_type,from_team,time
0,0,3,radiant,1
1,1,3,radiant,222
2,2,3,dire,143
3,3,3,radiant,143
4,4,3,dire,53


Видимо, это какое-то событие, их происходит в mid-е по одному-два раза

#### Как признаки будем использовать количество денег, которое игроки успели получить к 10-ой минуте.

In [95]:
gold = gold[gold['times']==600]

radiant_gold = gold[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].sum(axis=1)
dire_gold = gold[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].sum(axis=1)

diff_gold = radiant_gold - dire_gold
ratio_gold = radiant_gold / dire_gold

#### XP & LH

In [96]:
xp = xp[xp['times']==600]

radiant_xp = xp[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].sum(axis=1)
dire_xp = xp[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].sum(axis=1)

diff_xp = radiant_xp - dire_xp
ratio_xp = radiant_xp / dire_xp

In [100]:
lh = lh[lh['times']==600]

radiant_lh = lh[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].sum(axis=1)
dire_lh = lh[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].sum(axis=1)

diff_lh = radiant_lh - dire_lh
ratio_lh = radiant_lh / dire_lh

In [110]:
data = pd.DataFrame({'mid':np.arange(49948),
                     'radiant_gold':radiant_gold,
                     'dire_gold':dire_gold,
                     'diff_gold':diff_gold,
                     'ratio_gold':ratio_gold,
                     'radiant_xp':radiant_xp,
                     'dire_xp':dire_xp,
                     'diff_xp':diff_xp,
                     'ratio_xp':ratio_xp,
                     'radiant_lh':radiant_lh,
                     'dire_lh':dire_lh,
                     'diff_lh':diff_lh,
                     'ratio_lh':ratio_lh})
data.head()

,diff_gold,diff_lh,diff_xp,dire_gold,dire_lh,dire_xp,mid,radiant_gold,radiant_lh,radiant_xp,ratio_gold,ratio_lh,ratio_xp
9,-641,-44,926,22095,192,14930,0,21454,148,15856,0.970989,0.770833,1.062023
19,-2371,13,-2106,24536,144,17337,1,22165,157,15231,0.903366,1.090278,0.878526
29,5844,75,5558,15548,99,12445,2,21392,174,18003,1.375868,1.757576,1.446605
39,3011,-11,1609,17617,143,13933,3,20628,132,15542,1.170914,0.923077,1.115481
49,554,42,2258,17484,101,13076,4,18038,143,15334,1.031686,1.415842,1.172683


In [134]:
data = pd.DataFrame({'mid':np.arange(49948),
                     'radiant_gold':radiant_gold,
                     'dire_gold':dire_gold,
                     'diff_gold':diff_gold,
                     'ratio_gold':ratio_gold,})
data.head()

,diff_gold,dire_gold,mid,radiant_gold,ratio_gold
9,-641,22095,0,21454,0.970989
19,-2371,24536,1,22165,0.903366
29,5844,15548,2,21392,1.375868
39,3011,17617,3,20628,1.170914
49,554,17484,4,18038,1.031686


In [135]:
train = pd.merge(train_matches[['mid']], data, on='mid', how='left').drop(['mid'], 1)
test = pd.merge(test_matches[['mid']], data, on='mid', how='left').drop(['mid'], 1)

x_train = train.values
y_train = train_matches.radiant_won.values
x_test = test.values


0.63740423688453518

In [136]:
logreg = LogisticRegression()
np.mean(cross_val_score(logreg, x_train, y_train, cv=5, scoring='roc_auc'))

0.70548990458441474

In [138]:
logreg.fit(x_train, y_train)
test_matches['radiant_won'] = logreg.predict_proba(x_test)
test_matches.to_csv('baseline1.csv', index=None)

In [139]:

clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=1234)
np.mean(cross_val_score(clf, x_train, y_train, cv=5, scoring='roc_auc'))
clf.fit(x_train, y_train)
test_matches['radiant_won'] = clf.predict_proba(x_test)
test_matches.to_csv('baseline2.csv', index=None)